In [1]:
from scipy.linalg import qr, pinv, solve, norm
from numpy.random import randn
from numpy.linalg import lstsq
import numpy as np

In [3]:
# generate random data matrix
n,d = 6,4
X = randn(n,d)

# optional: give it linearly dependent columns
# X[:,3] = X[:,2]

# Understanding the pseudoinverse

In [4]:
# form pseudoinverse
Xd = pinv(X)

In [5]:
# X†X ≈ I_d
Xd @ X

array([[ 1.00000000e+00,  3.26194267e-16, -2.32599523e-16,
        -1.76538379e-16],
       [ 1.23838370e-16,  1.00000000e+00, -7.15454140e-16,
         3.20159051e-16],
       [-2.99593137e-16, -2.24746860e-16,  1.00000000e+00,
         1.57207172e-16],
       [-8.35114587e-17, -2.19207791e-16,  3.92819212e-17,
         1.00000000e+00]])

In [6]:
np.allclose(Xd @ X, np.identity(4))

True

In [7]:
# XX† !≈ I_n
X @ Xd

array([[ 0.92292723, -0.19207847, -0.02487266, -0.02034622,  0.15734442,
        -0.0919159 ],
       [-0.19207847,  0.29368018,  0.04785243, -0.24610263,  0.05077907,
        -0.32419612],
       [-0.02487266,  0.04785243,  0.93897132,  0.08772047,  0.21549265,
         0.01623931],
       [-0.02034622, -0.24610263,  0.08772047,  0.82689984, -0.25147931,
        -0.10592118],
       [ 0.15734442,  0.05077907,  0.21549265, -0.25147931,  0.16689241,
         0.04499583],
       [-0.0919159 , -0.32419612,  0.01623931, -0.10592118,  0.04499583,
         0.85062902]])

In [8]:
np.allclose(X @ Xd, np.identity(6))

False

In [9]:
Q,R = qr(X)
Q,R = qr(X, mode='economic')

In [10]:
np.allclose(X, Q @ R)

True

In [11]:
Q

array([[-0.56201486,  0.63759906, -0.23057605, -0.38388624],
       [-0.14763691, -0.21633913,  0.46831834,  0.07588703],
       [-0.27210148, -0.70101235, -0.21471219, -0.57219968],
       [ 0.66267283,  0.14334266, -0.15547092, -0.58570148],
       [-0.38277792, -0.12482775, -0.06724561, -0.01641728],
       [ 0.05147032, -0.13826577, -0.80790972,  0.41969547]])

In [12]:
R

array([[ 2.82036938, -0.4006528 ,  1.90362682,  0.34118195],
       [ 0.        ,  2.55806078, -0.13829193, -0.39913027],
       [ 0.        ,  0.        , -3.47705222,  0.39879886],
       [ 0.        ,  0.        ,  0.        , -1.72866855]])

In [13]:
print(np.allclose(Q.T @ Q, np.identity(Q.shape[1])))
Q.T @ Q

True


array([[ 1.00000000e+00, -7.06106860e-17,  1.63377044e-17,
        -4.64978195e-17],
       [-7.06106860e-17,  1.00000000e+00,  9.28390587e-17,
        -4.27972210e-17],
       [ 1.63377044e-17,  9.28390587e-17,  1.00000000e+00,
         3.18741738e-17],
       [-4.64978195e-17, -4.27972210e-17,  3.18741738e-17,
         1.00000000e+00]])

In [18]:
# form data from noisy linear model
wtrue = randn(d)
y = X.dot(wtrue) + .01*randn(n)

In [26]:
# solve least squares problem to estimate w
Q,R = qr(X, mode='economic')
w = solve(R, Q.T @ y)

0.0015392303466796875

In [20]:
# how good is our estimate?
norm(w - wtrue)

0.013572977104475286

In [21]:
# compute mean square error
def mse(y,z):
    return sum((y-z)**2)/len(y)
    
mse(y,X.dot(w))

6.898214932602962e-05

In [22]:
# we can use the numpy.lstsq call instead
w_lstsq = np.linalg.lstsq(X, y, rcond=None)[0]
norm(w_lstsq - w)

1.025079205553247e-15

# Compute QR by hand

In [18]:
n,d = X.shape 
X0 = X.copy()
R = np.zeros((n,d))
Q = np.zeros((n,n))

# first column of Q points in direction of first column of X
r = norm(X[:,0])
Q[:,0] = X[:,0]/r
Q

array([[-0.47534812,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.02906357,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.00071364,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.45358044,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-0.26950649,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-0.70344154,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ]])

In [19]:
# ensure Q*R matches X on first column
R[0,0] = r

In [20]:
# verify Q*R matches X in first column
(Q@R - X)[:,0]

array([0., 0., 0., 0., 0., 0.])

In [21]:
# now delete that part from X; we've covered it already
X[:,0] -= Q[:,0]*R[0,0]

In [22]:
# verify Q*R + X = X0
np.isclose(Q@R + X, X0)

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [23]:
# eliminate component of other columns in direction of first column of Q 
for j in range(1,d):
    R[0,j] = Q[:,0].dot(X[:,j])
    X[:,j] -= Q[:,0]*R[0,j]
R

array([[ 3.74544392, -0.69063861,  0.96961492,  0.82682362],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [24]:
# verify Q*R + X = X0
np.isclose(Q@R + X, X0)

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [25]:
# now for all the columns!
X = X0.copy()
Q *= 0
R *= 0

# compute the QR decomposition
for i in range(d):
    r = norm(X[:,i])
    Q[:,i] = X[:,i]/r
    for j in range(i,d):
        R[i,j] = Q[:,i].dot(X[:,j])
        X[:,j] -= Q[:,i]*R[i,j]
    print("iteration",i,": QR + X = X0?", np.isclose(Q@R + X, X0).all())

iteration 0 : QR + X = X0? True
iteration 1 : QR + X = X0? True
iteration 2 : QR + X = X0? True
iteration 3 : QR + X = X0? True


In [26]:
"""Our very own QR function to compute the economy QR"""
def ourQR(X0):
    X = X0.copy()
    n,d = X.shape
    R = np.zeros((n,d))
    Q = np.zeros((n,n))

    # compute the QR decomposition
    for i in range(d):
        r = norm(X[:,i])
        Q[:,i] = X[:,i]/r
        for j in range(i,d):
            R[i,j] = Q[:,i].dot(X[:,j])
            X[:,j] -= Q[:,i]*R[i,j]
    return Q,R

In [31]:
# solve least squares problem to estimate w
Q,R = ourQR(X0)
w_byhand = solve(R[:d,:d], (Q.T @ y)[:d])

In [32]:
norm(w_byhand - w)

1.3822165187958571e-15